# Exercise 3

Check if ffmpeg is installed, if not install it using [Homebrew](https://brew.sh/):

In [1]:
exist = !which ffmpeg
if not exist:
    !brew install ffmpeg

In [35]:
import glob
from math import gcd
from pathlib import Path
import subprocess
import json

## Helper functions

In [3]:
def calculate_aspect_ratio(width:int, height:int) -> str:
    """Calculate the video aspect ratio from width and height."""
    
    # Find the greatest common divisor of the width and height
    divisor = gcd(width, height)
    
    # Calculate the simplest form of the aspect ratio
    x = width // divisor
    y = height // divisor
    
    # Return a string representation of the aspect ratio
    return f"{x}:{y}"

In [4]:
def bit_rate_to_mbps(bit_rate:int) -> str:
    """Convert a bit rate in bits per second to megabits per second."""
    
    # Convert the bit rate to megabits per second
    mbps = bit_rate / 1000000
    
    # Return a string representation of the megabits per second
    return f"{mbps} Mb/s"

In [5]:
def bit_rate_to_kbps(bit_rate:int) -> str:
    """Convert a bit rate in bits per second to kilobits per second."""
    
    # Convert the bit rate to kilobits per second
    kbps = bit_rate / 1000
    
    # Return a string representation of the kilobits per second
    return f"{kbps} Kb/s"

In [43]:
def ffprobe(file_path):
    """Get video metadata using ffprobe."""
    
    # Run ffprobe on the file path
    command = ['ffprobe',
               '-show_format',
               '-show_streams',
               '-of', 'json',
               file_path]
    process = subprocess.Popen(command, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    out, err = process.communicate()

    # Return the result
    return json.loads(out.decode('utf-8'))

## Process files

In [44]:
def is_file_valid(file_path):
    valid = True

    probe_data = ffprobe(file_path)
    print("file: " + file_path)

    # extract metadata about the video stream
    video_stream_info = next((stream for stream in probe_data['streams'] if stream['codec_type'] == 'video'), None)

    # extract metadata about the audio stream
    audio_stream_info = next((stream for stream in probe_data['streams'] if stream['codec_type'] == 'audio'), None)

    # check if the container format is mp4
    if not probe_data['format']['format_name'] == "mov,mp4,m4a,3gp,3g2,mj2":
        print("Wrong container format: " + probe_data['format']['format_name'])
        valid = False
    
    # check if the video codec is h264
    if not video_stream_info['codec_name'] == "h264":
        print("Wrong video codec: " + probe_data['streams'][0]['codec_name'])
        valid = False
    
    # check if the audio codec is aac
    if not audio_stream_info['codec_name'] == "aac":
        print("Wrong audio codec: " + probe_data['streams'][1]['codec_name'])
        valid = False

    # check if frame rate is 25
    if not video_stream_info['r_frame_rate'] == "25/1":
        print("Wrong frame rate: " + video_stream_info['r_frame_rate'])
        valid = False
    
    # check if the video aspect ratio is 16:9
    aspect_ratio = video_stream_info.get('display_aspect_ratio')
    if not aspect_ratio:
        aspect_ratio = calculate_aspect_ratio(int(video_stream_info['width']), int(video_stream_info['height']))

    if not aspect_ratio == "16:9":
        print("Wrong aspect ratio: " + aspect_ratio)
    
    # check if the video resolution is 640x360
    if not int(video_stream_info['width']) == 640 or not int(video_stream_info['height']) == 360:
        print(f"Wrong resolution: {video_stream_info['width']}x{video_stream_info['height']}")
        valid = False

    # check if the video bitrate is 2 – 5 Mb/s
    video_bit_rate = int(video_stream_info['bit_rate'])
    if not 2000000 <= video_bit_rate <= 5000000:
        print("Wrong video bitrate: " + bit_rate_to_mbps(video_bit_rate))
        valid = False

    # check if the audio bitrate is up to 256 kb/s
    audio_bit_rate = int(audio_stream_info['bit_rate'])
    if not audio_bit_rate <= 256000:
        print("Wrong audio bitrate: " + bit_rate_to_kbps(audio_bit_rate))
        valid = False

    # check if the audio channel is stereo
    if not audio_stream_info['channels'] == 2:
        print("Audio channel is not stereo, a number of channels: " + str(audio_stream_info['channels']))
        valid = False
    
    if valid:
        print("File is VALID.\n")
    else:
        print("File is INVALID.\n")
    
    return valid

In [45]:
invalid_files = []

for file_path in glob.glob("Exercise3_Films/*"):
    if not is_file_valid(file_path):
        invalid_files.append(file_path)



file: Exercise3_Films/Last_man_on_earth_1964.mov
Wrong video codec: prores
Wrong audio codec: pcm_s16le
Wrong frame rate: 24000/1001
Wrong video bitrate: 9.285191 Mb/s
Wrong audio bitrate: 1536.0 Kb/s
File is INVALID.

file: Exercise3_Films/Voyage_to_the_Planet_of_Prehistoric_Women.mp4
Wrong video codec: hevc
Wrong audio codec: mp3
Wrong frame rate: 30000/1001
Wrong video bitrate: 8.038857 Mb/s
Wrong audio bitrate: 320.0 Kb/s
File is INVALID.

file: Exercise3_Films/The_Gun_and_the_Pulpit.avi
Wrong container format: avi
Wrong video codec: rawvideo
Wrong audio codec: pcm_s16le
Wrong aspect ratio: 180:101
Wrong resolution: 720x404
Wrong video bitrate: 87.438878 Mb/s
Wrong audio bitrate: 1536.0 Kb/s
File is INVALID.

file: Exercise3_Films/Cosmos_War_of_the_Planets.mp4
Wrong frame rate: 30000/1001
Wrong aspect ratio: 314:177
Wrong resolution: 628x354
Wrong audio bitrate: 317.103 Kb/s
File is INVALID.

file: Exercise3_Films/The_Hill_Gang_Rides_Again.mp4
Wrong video bitrate: 7.53773 Mb/s
File

In [46]:
for file_path in invalid_files:
    print("Reencoding file: " + file)

    # make directory if not exists
    Path("results").mkdir(parents=True, exist_ok=True)

    # run ffmpeg command
    command = [
        'ffmpeg',
        '-i', file_path, # input file

        # use a complex filter to scale the video stream to 640x360 and set the frame rate to 25
        '-filter_complex', '[0:v]scale=height=360:width=640[s0];[s0]fps=fps=25:round=up[s1]',

        # audio options
        '-map', '0:a', # use the audio stream
        '-acodec', 'aac', # set audio codec
        '-ab', '256k', # set audio bitrate
        '-ac', '2', # set audio channel (stereo)

        # video options
        '-map', '[s1]', # use the scaled video stream
        '-aspect', '16:9', # set video aspect ratio
 
        # set video codec
        '-vcodec', 'h264', 

        # set video bitrate (2 – 5 Mb/s) 
        '-maxrate', '5M', # set max video bitrate
        '-minrate', '2M', # set min video bitrate
        '-bufsize', '5M', # set buffer size, because of maxrate doesn't work without this
        '-x264-params', '"nal-hrd=cbr"', # set constant bitrate mode, because of minrate don't work without this
        '-b:v', '3M', # set average video bitrate

        '-y', # overwrite output file if it exists
        "results/" + Path(file_path).stem + "_formatOK.mp4" # output file
    ]

    returncode = subprocess.run(command).returncode
    assert returncode == 0

print("\n")

Reencoding file: Exercise3_Films/The_Hill_Gang_Rides_Again.mp4


ffmpeg version 6.0 Copyright (c) 2000-2023 the FFmpeg developers
  built with Apple clang version 14.0.3 (clang-1403.0.22.14.1)
  configuration: --prefix=/opt/homebrew/Cellar/ffmpeg/6.0_1 --enable-shared --enable-pthreads --enable-version3 --cc=clang --host-cflags= --host-ldflags= --enable-ffplay --enable-gnutls --enable-gpl --enable-libaom --enable-libaribb24 --enable-libbluray --enable-libdav1d --enable-libjxl --enable-libmp3lame --enable-libopus --enable-librav1e --enable-librist --enable-librubberband --enable-libsnappy --enable-libsrt --enable-libsvtav1 --enable-libtesseract --enable-libtheora --enable-libvidstab --enable-libvmaf --enable-libvorbis --enable-libvpx --enable-libwebp --enable-libx264 --enable-libx265 --enable-libxml2 --enable-libxvid --enable-lzma --enable-libfontconfig --enable-libfreetype --enable-frei0r --enable-libass --enable-libopencore-amrnb --enable-libopencore-amrwb --enable-libopenjpeg --enable-libspeex --enable-libsoxr --enable-libzmq --enable-libzimg --di

Reencoding file: Exercise3_Films/The_Hill_Gang_Rides_Again.mp4


frame=  501 fps=163 q=-1.0 Lsize=    7610kB time=00:00:20.03 bitrate=3111.9kbits/s speed=6.51x    
video:7000kB audio:593kB subtitle:0kB other streams:0kB global headers:0kB muxing overhead: 0.215589%
[aac @ 0x135e10020] Qavg: 52162.027
[libx264 @ 0x135e111b0] frame I:5     Avg QP: 5.20  size: 49626
[libx264 @ 0x135e111b0] frame P:128   Avg QP: 7.86  size: 32535
[libx264 @ 0x135e111b0] frame B:368   Avg QP:11.64  size:  7485
[libx264 @ 0x135e111b0] consecutive B-frames:  1.4%  0.8%  3.6% 94.2%
[libx264 @ 0x135e111b0] mb I  I16..4: 18.6% 38.0% 43.3%
[libx264 @ 0x135e111b0] mb P  I16..4:  3.8% 19.5%  9.5%  P16..4: 17.5% 21.8% 17.8%  0.0%  0.0%    skip:10.0%
[libx264 @ 0x135e111b0] mb B  I16..4:  0.1%  0.9%  0.5%  B16..8: 38.2% 19.4% 10.6%  direct: 7.5%  skip:22.8%  L0:46.3% L1:38.6% BI:15.0%
[libx264 @ 0x135e111b0] final ratefactor: 8.85
[libx264 @ 0x135e111b0] 8x8 transform intra:57.4% inter:46.4%
[libx264 @ 0x135e111b0] coded y,uvDC,uvAC intra: 86.4% 95.1% 94.8% inter: 36.0% 57.0% 31.6

Reencoding file: Exercise3_Films/The_Hill_Gang_Rides_Again.mp4


frame=  500 fps=155 q=-1.0 Lsize=    7709kB time=00:00:19.98 bitrate=3159.4kbits/s speed=6.19x    
video:7079kB audio:615kB subtitle:0kB other streams:0kB global headers:0kB muxing overhead: 0.208226%
[aac @ 0x1378086e0] Qavg: 21589.027
[libx264 @ 0x137809820] frame I:17    Avg QP: 9.68  size: 35016
[libx264 @ 0x137809820] frame P:156   Avg QP:12.03  size: 23518
[libx264 @ 0x137809820] frame B:327   Avg QP:14.37  size:  9124
[libx264 @ 0x137809820] consecutive B-frames: 10.0%  5.6%  8.4% 76.0%
[libx264 @ 0x137809820] mb I  I16..4: 10.2% 43.0% 46.8%
[libx264 @ 0x137809820] mb P  I16..4:  3.1% 31.6% 17.2%  P16..4: 14.2% 19.6% 12.6%  0.0%  0.0%    skip: 1.8%
[libx264 @ 0x137809820] mb B  I16..4:  0.5%  6.4%  4.7%  B16..8: 31.8% 24.3%  9.8%  direct: 6.2%  skip:16.3%  L0:48.7% L1:37.4% BI:13.9%
[libx264 @ 0x137809820] final ratefactor: 9.88
[libx264 @ 0x137809820] 8x8 transform intra:57.1% inter:48.6%
[libx264 @ 0x137809820] coded y,uvDC,uvAC intra: 89.6% 91.8% 80.8% inter: 44.8% 37.5% 15.5

Reencoding file: Exercise3_Films/The_Hill_Gang_Rides_Again.mp4


frame=  501 fps=157 q=-1.0 Lsize=    7852kB time=00:00:20.01 bitrate=3214.5kbits/s speed=6.28x    
video:7234kB audio:601kB subtitle:0kB other streams:0kB global headers:0kB muxing overhead: 0.208881%
[aac @ 0x142e36c80] Qavg: 43692.434
[libx264 @ 0x142e35150] frame I:3     Avg QP: 9.77  size: 38265
[libx264 @ 0x142e35150] frame P:126   Avg QP: 9.94  size: 31566
[libx264 @ 0x142e35150] frame B:372   Avg QP:13.08  size:  8911
[libx264 @ 0x142e35150] consecutive B-frames:  0.8%  0.4%  0.6% 98.2%
[libx264 @ 0x142e35150] mb I  I16..4: 10.4% 64.8% 24.9%
[libx264 @ 0x142e35150] mb P  I16..4:  4.4% 28.4%  8.0%  P16..4: 14.1% 26.1% 18.5%  0.0%  0.0%    skip: 0.5%
[libx264 @ 0x142e35150] mb B  I16..4:  0.8%  3.3%  0.8%  B16..8: 32.7% 28.3% 12.3%  direct:11.5%  skip:10.3%  L0:41.5% L1:39.3% BI:19.2%
[libx264 @ 0x142e35150] final ratefactor: 10.01
[libx264 @ 0x142e35150] 8x8 transform intra:69.0% inter:46.4%
[libx264 @ 0x142e35150] coded y,uvDC,uvAC intra: 88.9% 97.7% 96.9% inter: 51.0% 63.1% 32.

Reencoding file: Exercise3_Films/The_Hill_Gang_Rides_Again.mp4


frame=  500 fps=161 q=-1.0 Lsize=    7790kB time=00:00:19.99 bitrate=3191.9kbits/s speed=6.45x    
video:7249kB audio:523kB subtitle:0kB other streams:0kB global headers:0kB muxing overhead: 0.228534%
[aac @ 0x11ff22c20] Qavg: 64195.047
[libx264 @ 0x11ff23c30] frame I:4     Avg QP: 6.22  size: 42502
[libx264 @ 0x11ff23c30] frame P:130   Avg QP: 6.57  size: 36340
[libx264 @ 0x11ff23c30] frame B:366   Avg QP:10.22  size:  6907
[libx264 @ 0x11ff23c30] consecutive B-frames:  2.0%  0.8%  1.2% 96.0%
[libx264 @ 0x11ff23c30] mb I  I16..4: 12.6% 36.5% 50.9%
[libx264 @ 0x11ff23c30] mb P  I16..4:  2.6% 19.9% 14.6%  P16..4: 17.0% 23.8% 20.4%  0.0%  0.0%    skip: 1.6%
[libx264 @ 0x11ff23c30] mb B  I16..4:  0.2%  0.8%  0.4%  B16..8: 36.5% 20.8% 12.5%  direct: 8.7%  skip:20.0%  L0:44.0% L1:40.5% BI:15.5%
[libx264 @ 0x11ff23c30] final ratefactor: 7.36
[libx264 @ 0x11ff23c30] 8x8 transform intra:52.8% inter:35.7%
[libx264 @ 0x11ff23c30] coded y,uvDC,uvAC intra: 96.2% 89.0% 86.9% inter: 43.8% 43.9% 23.0

Check reencoded files:

In [29]:
for file_path in glob.glob("results/*"):
    is_file_valid(file_path)

file: results/Last_man_on_earth_1964_formatOK.mp4
File is VALID.

file: results/Voyage_to_the_Planet_of_Prehistoric_Women_formatOK.mp4
File is VALID.

file: results/Cosmos_War_of_the_Planets_formatOK.mp4
File is VALID.

file: results/The_Hill_Gang_Rides_Again_formatOK.mp4
File is VALID.

file: results/The_Gun_and_the_Pulpit_formatOK.mp4
File is VALID.

